In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda1,floatX=float32, nvcc.flags=-D_FORCE_INLINES"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from __future__ import print_function
import keras
import os.path

import densenet
import numpy as np
import sklearn.metrics as metrics
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.models import model_from_json
import json
from inner_func import get_hubs_last_conv_name, get_filtered_idx, get_last_conv_layer_name, recursive_find_root_conv
import numpy as np
from os import listdir

batch_size = 100
nb_classes = 38
nb_epoch = 300

img_rows, img_cols = 224, 224
img_channels = 3

img_dim = (img_rows, img_cols, img_channels) 
nb_classes=38
depth = 169
nb_dense_block = 4
nb_layers_per_block=[6, 12, 32, 32]
growth_rate = 32
nb_filter = 64

dropout_rate = 0.5 # 0.0 for data augmentation
root_total = '/media/data/TOO/DATASETS/PlantVillage/PlantVillage/train/'
disease_CLASSES = listdir(root_total)

In [ ]:
import sys
sys.setrecursionlimit(3000)
#from keras.utils import plot_model
batch_size = 48
nb_epoch = 30

    # Load our model
#model = TELU_DEPTH.DenseNetImageNet121(img_dim, classes=nb_classes, reduction=0.5, 
#                           dropout_rate=dropout_rate, bottleneck=True,weights=None)
model=densenet.DenseNet(img_dim, depth=depth, nb_dense_block=nb_dense_block, growth_rate=growth_rate, nb_filter=nb_filter,
                    nb_layers_per_block=nb_layers_per_block, bottleneck=True, reduction=0.5,
                    dropout_rate=dropout_rate, subsample_initial_block=True,
                     weights=None, 
                    classes=nb_classes, activation='softmax')

print("Model created")
# Load model
weights_file='/media/data/TOO/plantsdiseases/imagenet_models/densenet169_weights_tf.h5'

model.load_weights(weights_file, by_name=True)
print("Model loaded.")

model.summary()

In [ ]:
print(keras.__version__)

In [ ]:
def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (img_rows, img_cols), interpolation=cv2.INTER_LINEAR)
    return resized

def load_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    
    for fld in disease_CLASSES:
        index = disease_CLASSES.index(fld)
        
        path = os.path.join('..', 'DATASETS', 'train', fld,'*')
        files = glob.glob(path)
        #print path
        #print files
        for fl in files:
            #print str(fl)
            flbase = os.path.basename(fl)
            img = get_im_cv2(fl)
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(index)

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id

train_data, train_target, train_id = load_train()

train_data=np.array(train_data, dtype=np.uint8)

train_data=(train_data.astype('float32'))/255

#mean and std deviation of training set
data_mean=np.mean(train_data,axis=0)
data_stddev=np.std(train_data,axis=0)

#normalize data according to mean and sdev of training set
train_data=(train_data-data_mean)/data_stddev

#Reshaping the dataset
train_data=train_data.transpose((0, 1, 2, 3))

train_target = np_utils.to_categorical(train_target, nb_classes)

print('train_data shape:', train_data.shape)
print(train_data.shape[0], 'train samples')




In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val, Y_train,Y_val=train_test_split(train_data,train_target,test_size=0.2, random_state=42)

In [ ]:
model.summary()

sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy','top_k_categorical_accuracy'])
hist=model.fit(x_train, Y_train,
              batch_size=48,
              epochs=100,
              shuffle=True,
              verbose=1,
              validation_data=(x_val,Y_val),
              )



In [ ]:
# summarize history for accuracy
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(hist.history['acc']); plt.plot(hist.history['val_acc']);
plt.title('model accuracy'); plt.ylabel('Accuracy');
plt.xlabel('Epoch'); plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='upper left');
# summarize history for Top_5_accuracy
plt.show()
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(hist.history['top_k_categorical_accuracy']); plt.plot(hist.history['val_top_k_categorical_accuracy']);
plt.title('model Top_5 Accuracy'); plt.ylabel('Accuracy');
plt.xlabel('Epoch'); plt.legend(['Top_5 Training Accuracy', 'Top_5 Validation Accuracy'], loc='upper left');
# summarize history for loss
plt.show()
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(hist.history['loss']); plt.plot(hist.history['val_loss']);
plt.title('model loss'); plt.ylabel('Loss');
plt.xlabel('Epoch'); plt.legend(['Training Loss', 'Validation Loss'], loc='upper left');
plt.show()

In [ ]:
import numpy as np
np.random.seed(1337)
import glob
import time
import cv2
def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (img_rows, img_cols), interpolation=cv2.INTER_LINEAR)
    return resized
def load_test():
    X_test = []
    X_test_id = []
    y_test = []
    start_time = time.time()

    print('Read TEST images')
    #diseaseNames =  ['0']
  
    for fld in disease_CLASSES:
        index = disease_CLASSES.index(fld)
        
        path = os.path.join('..', 'DATASETS', 'test', fld,'*')
        
        files = glob.glob(path)
        #print (path)
        #print (files)
        for fl in files:
            #print str(fl)
            flbase = os.path.basename(fl)
            img = get_im_cv2(fl)
            X_test.append(img)
            X_test_id.append(flbase)
            y_test.append(index)

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_test, y_test, X_test_id
test_data, test_target,test_id = load_test()
X_test = np.array(test_data, dtype=np.uint8)
#mean and std deviation of training set
X_test = (X_test.astype('float32'))/255
test_mean=np.mean(X_test,axis=0)
test_stddev=np.std(X_test,axis=0)



#normalize data according to mean and sdev of training set

X_test=(X_test-test_mean)/test_stddev

X_test = X_test.transpose((0, 1, 2, 3))

# Normalizing the data

#X_test /= 255
y_test = np_utils.to_categorical(test_target, nb_classes)
print(X_test.shape[0], 'test samples')

In [ ]:
score=model.evaluate(X_test,y_test,verbose=1)

print('Test Loss:',score[0])
print('Test Accuracy:',score[1]*100)
print ('Top_5 Accuracy',score[2]*100)
print('Top_5 Error',100-score[2]*100)